In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd
import datetime


!pip install locationtagger

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
import locationtagger

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
import pycountry
countries = {}
for con in pycountry.countries:
  countries[con.alpha_2] = con.name

In [ ]:
old_data_path = '/content/drive/My Drive/capstone_data/data/1_pre-Ngram_processed'
# countries = ['AU','BD','CA','GB','GH','HK','IE','IN','JM','KE','LK','MY','NG','NZ','PH','PK','SG','TZ','US','ZA']

In [ ]:
# 'In' 'NG'
country = 'NG'
filename = country + '_pre-preprocess.csv'
path = os.path.join(old_data_path, filename)
df = pd.read_csv(path, index_col=[0])
df.head()

,article_id,article_title,publisher,year,article_text
0,51128383,OMSL: SAA Doesn't Constitute Threat to Nationa...,thestreetjournal.org,2019,OMSL SAA Doesn't Constitute Threat to Nationa...
1,51083517,EFCC Grills Zamfara INEC Officials over Divers...,thestreetjournal.org,2019,EFCC Grills Zamfara INEC Officials over Diver...
2,51247758,Alleged N1.4bn Fraud: How Usoro was Paid N700m...,thestreetjournal.org,2019,Alleged N1.4bn Fraud How Usoro was Paid N700m...
3,51123139,Opinion: Adesuwa of Aruwa Capital Management: ...,thestreetjournal.org,2019,Opinion Adesuwa of Aruwa Capital Management A...
4,51070131,Kidnappers of NSCDC Officials Demand N6m Ranso...,thestreetjournal.org,2019,Kidnappers of NSCDC Officials Demand N6m Rans...


In [ ]:
df.shape[0]

79270

In [ ]:

counter =0
a = datetime.datetime.now()
def country_ner(text):
  if len(text)>1000000:
    return []

  entities = locationtagger.find_locations(text = text)
  
  global counter
  counter +=1

  if counter % 100 == 0:  # i.e. print in multiples of 500
    b = datetime.datetime.now()
    print("num files: {}, time took so far: {}, % completed: {:0.4f}% ".format(counter, b-a, counter/df.shape[0]*100.0))

  return entities.country_mentions



def isDomestic(row, country):

  country_list = list(dict(row['country_mention']).keys())
  if len(country_list) > 0:
    if country not in country_list:
      return False 

  return True 


In [ ]:
# df

In [ ]:
df['country_mention'] = df.article_text.apply(country_ner)

num files: 100, time took so far: 0:00:39.315892, % completed: 10.5042% 


KeyboardInterrupt: ignored

In [ ]:
country_name = countries.get(country)
df['Domestic'] = df.apply(lambda row: isDomestic(row, country_name),axis=1)

In [ ]:
# df.Domestic.value_counts(normalize=True) 

True     0.706
False    0.294
Name: Domestic, dtype: float64

In [ ]:
df_domestic = df[df.Domestic == True]
df_international = df[df.Domestic == False]

new_data_path = '/content/drive/My Drive/capstone_data/data/2_domestic_filter'

domestic_output_filename = country + '_domestic.csv'
international_output_filename = country + '_international.csv'

df_domestic.to_csv(os.path.join(new_data_path, domestic_output_filename))
df_international.to_csv(os.path.join(new_data_path, international_output_filename))